In [194]:
import polars as pl
import polars.selectors as cs
from polars import functions as pf
import math
import os

In [3]:
COMPANY_NAME_LIST = ['Apple%2520Inc.%2520(AAPL)%2520(CIK%25200000320193)', 'MICROSOFT%2520CORP%2520(MSFT)%2520(CIK%25200000789019)', 
                     'BigCommerce%2520Holdings%252C%2520Inc.%2520(BIGC)%2520(CIK%25200001626450)', 'ROKU%252C%2520INC%2520(ROKU)%2520(CIK%25200001428439)', 
                     'JPMORGAN%2520CHASE%2520(CIK%25200000019617)', 'VISA%2520INC.%2520(V)%2520(CIK%25200001403161)', 
                     'Block%252C%2520Inc.%2520(SQ%252C%2520BSQKZ)%2520(CIK%25200001512673)', 'Robinhood%2520Markets%252C%2520Inc.%2520(HOOD)%2520(CIK%25200001783879)', 
                     'JOHNSON%252C%252C%2520JOHNSON%2520(JNJ)%2520(CIK%25200000200406)', 'PFIZER%2520INC%2520(PFE)%2520(CIK%25200000078003)', 
                     'Moderna%252C%2520Inc.%2520(MRNA)%2520(CIK%25200001682852)', 'Teladoc%2520Health%252C%2520Inc.%2520(TDOC)%2520(CIK%25200001477449)', 
                     'EXXON%2520MOBIL%2520CORP%2520(XOM)%2520(CIK%25200000034088)', 'CHEVRON%2520CORP%2520(CVX)%2520(CIK%25200000093410)', 
                     'FIRST%2520SOLAR%252C%2520INC.%2520(FSLR)%2520(CIK%25200001274494)', 'PLUG%2520POWER%2520INC%2520(PLUG)%2520(CIK%25200001093691)', 
                     'GENERAL%252CELECTRIC%2520CO%2520(GE)%2520(CIK%25200000040545)', '3M%2520CO%2520(MMM)%2520(CIK%25200000066740)', 
                     'CATERPILLAR%2520INC%2520(CAT)%2520(CIK%25200000018230)', 'FASTENAL%2520CO%2520(FAST)%2520(CIK%25200000815556)']

In [3]:
df = pl.read_parquet(r'C:\Users\310\Desktop\Progects_Py\cleared_files\PLUG%2520POWER%2520INC%2520(PLUG)%2520(CIK%25200001093691)_reports.parquet')

In [4]:
lm_dict = pl.read_csv(r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\Analysis\Loughran-McDonald_MasterDictionary_1993-2021.csv')
lm_dict

Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables,Source
str,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""AARDVARK""",1,354,1.5501e-8,1.4226e-8,0.000004,99,0,0,0,0,0,0,0,2,"""12of12inf"""
"""AARDVARKS""",2,3,1.3136e-10,8.6538e-12,9.2417e-9,1,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ABACI""",3,9,3.9409e-10,1.1697e-10,5.2905e-8,7,0,0,0,0,0,0,0,3,"""12of12inf"""
"""ABACK""",4,29,1.2698e-9,6.6547e-10,1.5951e-7,28,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ABACUS""",5,8570,3.7526e-7,3.8095e-7,0.000035,1108,0,0,0,0,0,0,0,3,"""12of12inf"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZYGOTE""",86529,50,2.1894e-9,8.7293e-10,1.8860e-7,35,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ZYGOTES""",86530,1,4.3788e-11,1.8095e-11,1.9324e-8,1,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ZYGOTIC""",86531,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,3,"""12of12inf"""


In [44]:
df_dict = {}

for company_name in COMPANY_NAME_LIST:

    clean_name = company_name.split('%')[0]
    print(clean_name)

    df_name = f"df_{clean_name}"

    file_path_partial = os.path.join(r"C:\Users\310\Desktop\Progects_Py\cleared_files", company_name)
    file_path = file_path_partial + '_reports.parquet'
    print(file_path)

    df = pl.read_parquet(file_path)

    df_dict[df_name] = df



Apple
C:\Users\310\Desktop\Progects_Py\cleared_files\Apple%2520Inc.%2520(AAPL)%2520(CIK%25200000320193)_reports.parquet
MICROSOFT
C:\Users\310\Desktop\Progects_Py\cleared_files\MICROSOFT%2520CORP%2520(MSFT)%2520(CIK%25200000789019)_reports.parquet
BigCommerce
C:\Users\310\Desktop\Progects_Py\cleared_files\BigCommerce%2520Holdings%252C%2520Inc.%2520(BIGC)%2520(CIK%25200001626450)_reports.parquet
ROKU
C:\Users\310\Desktop\Progects_Py\cleared_files\ROKU%252C%2520INC%2520(ROKU)%2520(CIK%25200001428439)_reports.parquet
JPMORGAN
C:\Users\310\Desktop\Progects_Py\cleared_files\JPMORGAN%2520CHASE%2520(CIK%25200000019617)_reports.parquet
VISA
C:\Users\310\Desktop\Progects_Py\cleared_files\VISA%2520INC.%2520(V)%2520(CIK%25200001403161)_reports.parquet
Block
C:\Users\310\Desktop\Progects_Py\cleared_files\Block%252C%2520Inc.%2520(SQ%252C%2520BSQKZ)%2520(CIK%25200001512673)_reports.parquet
Robinhood
C:\Users\310\Desktop\Progects_Py\cleared_files\Robinhood%2520Markets%252C%2520Inc.%2520(HOOD)%2520(CI

In [6]:
total_num_of_rep = 0

for value in df_dict.values():
    total_num_of_rep += value.width
print(total_num_of_rep)

289


In [7]:
total_num_of_words = 0

for value in df_dict.values():
    for col in value:
        total_num_of_words += col.count()

print(total_num_of_words)

10371991


In [169]:
#average doc length
average_word_count = total_num_of_words/total_num_of_rep
average_word_count

35889.242214532875

In [159]:
#ENSURE CASE CoMpAtAbIlItY
positive_words = lm_dict.filter(lm_dict["Positive"] > 0).to_series().str.to_lowercase()

negative_words = lm_dict.filter(lm_dict["Negative"] > 0).to_series().str.to_lowercase()
print(len(negative_words) + len(positive_words))



2692


In [160]:
#example series 
example_series = df_dict['df_3M'][df_dict['df_3M'].columns[1]]
example_df = df_dict['df_3M']


This is the funk, that appled to a series and get sentiment words fron it 

In [161]:
def count_sentiment_w_in_doc(df_series):
        
        #grouped series that became a df, where each row is unique word and its count in document 
        num_of_w = df_series.value_counts().rename({"count": "count in doc"})
       
        
        #first col of this df (only words by itself)
        only_w = num_of_w[num_of_w.columns[0]]
        

        #Filterd df, there is only words that have positive or negative sentiment score in this document 
        pos_w = num_of_w.filter(only_w.is_in(positive_words)).with_columns(pl.lit(True).alias("positive"))
        neg_w = num_of_w.filter(only_w.is_in(negative_words)).with_columns(pl.lit(False).alias("positive"))
       
        #Staked for one df for further operations 
        df_stacked = neg_w.vstack(pos_w)

        return df_stacked
    
check = count_sentiment_w_in_doc(df_dict['df_3M'][df_dict['df_3M'].columns[1]])  
print(check)

shape: (237, 3)
┌────────────┬──────────────┬──────────┐
│ 2019-07-26 ┆ count in doc ┆ positive │
│ ---        ┆ ---          ┆ ---      │
│ str        ┆ u32          ┆ bool     │
╞════════════╪══════════════╪══════════╡
│ defending  ┆ 20           ┆ false    │
│ sluggish   ┆ 2            ┆ false    │
│ declines   ┆ 6            ┆ false    │
│ claims     ┆ 134          ┆ false    │
│ interfere  ┆ 2            ┆ false    │
│ …          ┆ …            ┆ …        │
│ able       ┆ 16           ┆ true     │
│ leading    ┆ 12           ┆ true     │
│ enabled    ┆ 2            ┆ true     │
│ best       ┆ 14           ┆ true     │
│ achieve    ┆ 2            ┆ true     │
└────────────┴──────────────┴──────────┘


Func that count a doc with at least one occurense of the i
th word

In [163]:
def in_all_doc_occurence(check_str, df_dict):
    count = 0
    for df in df_dict.values():
        count += df.select((pl.all() == check_str).sum() > 0).sum_horizontal()[0]
        
    return count

In [164]:
stringg = "a"
in_all_doc_occurence(stringg, df_dict)

289

In [165]:
def total_occurence_count(df_series, df_dict):
   check = count_sentiment_w_in_doc(df_series).lazy()

   counts = check.with_columns(pl.col(check.columns[0]).map_elements(lambda x: in_all_doc_occurence(x, df_dict)).alias("count in all docs")).collect()
   return counts

In [249]:
res_df = total_occurence_count(example_series, df_dict)
print(res_df)

shape: (237, 4)
┌───────────────┬──────────────┬──────────┬───────────────────┐
│ 2019-07-26    ┆ count in doc ┆ positive ┆ count in all docs │
│ ---           ┆ ---          ┆ ---      ┆ ---               │
│ str           ┆ u32          ┆ bool     ┆ i64               │
╞═══════════════╪══════════════╪══════════╪═══════════════════╡
│ dismissing    ┆ 2            ┆ false    ┆ 54                │
│ sluggish      ┆ 2            ┆ false    ┆ 8                 │
│ restructuring ┆ 70           ┆ false    ┆ 145               │
│ intentional   ┆ 2            ┆ false    ┆ 20                │
│ allegations   ┆ 10           ┆ false    ┆ 129               │
│ …             ┆ …            ┆ …        ┆ …                 │
│ benefited     ┆ 4            ┆ true     ┆ 74                │
│ alliances     ┆ 6            ┆ true     ┆ 38                │
│ enable        ┆ 14           ┆ true     ┆ 149               │
│ innovator     ┆ 2            ┆ true     ┆ 25                │
│ positive      ┆ 2     

In [252]:
ln_average_word_count = math.log(average_word_count)
res_df = res_df.with_columns((1 + (pl.col(res_df.columns[1]).log())/(1 + ln_average_word_count))*((total_num_of_rep/pl.col(res_df.columns[3])).log()).alias("fff"))
#res_df = res_df.with_columns((1 + (pl.col(res_df.columns[1]).log())/(1 + ln_average_word_count))*((total_num_of_rep/pl.col(res_df.columns[3])).log()).alias("gg"))
res_df
#interim = res_df.filter(pl.col('positive') == True)
#interim.select(pl.col())

2019-07-26,count in doc,positive,count in all docs,literal
str,u32,bool,i64,f64
"""dismissing""",2,false,54,1.778652
"""sluggish""",2,false,8,3.803408
"""restructuring""",70,false,145,0.944751
"""intentional""",2,false,20,2.831832
"""allegations""",10,false,129,0.968284
…,…,…,…,…
"""benefited""",4,true,74,1.526759
"""alliances""",6,true,38,2.345269
"""enable""",14,true,149,0.814665


In [167]:
check_series = check[check.columns[0]]
print(check_series)

counts = check_series.map_elements(lambda x: in_all_doc_occurence(x, df_dict))

print(counts)

shape: (237,)
Series: '2019-07-26' [str]
[
	"defending"
	"sluggish"
	"declines"
	"claims"
	"interfere"
	…
	"able"
	"leading"
	"enabled"
	"best"
	"achieve"
]
shape: (237,)
Series: '2019-07-26' [i64]
[
	97
	8
	156
	245
	18
	…
	192
	148
	48
	180
	162
]


General loop that iterates over dfs and over it columns

In [172]:
for df in df_dict.values():
    for col in df:
        total_occurence_count(col, df_dict)
        

(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(30175,)
(15261,)
(15261,)
(15261,)
(15261,)
(15261,)
(15261,)
(15261,)
(15261,)
(15261,)


PanicException: python function failed KeyboardInterrupt: 

PanicException: python function failed KeyboardInterrupt: 